# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 1 2022 8:30AM,245474,10,0085955024,ISDIN Corporation,Released
1,Sep 1 2022 8:30AM,245474,10,0085955025,ISDIN Corporation,Released
2,Sep 1 2022 8:30AM,245474,10,0085955052,ISDIN Corporation,Released
3,Sep 1 2022 8:30AM,245474,10,0085955053,ISDIN Corporation,Released
4,Sep 1 2022 8:30AM,245474,10,0085955059,ISDIN Corporation,Released
5,Sep 1 2022 8:30AM,245474,10,0085955060,ISDIN Corporation,Released
6,Sep 1 2022 8:30AM,245474,10,0085955061,ISDIN Corporation,Released
7,Aug 31 2022 5:24PM,245480,15,8467370,"Mizner Bioscience, LLC",Released
8,Aug 31 2022 5:24PM,245480,15,8467371,"Mizner Bioscience, LLC",Released
9,Aug 31 2022 5:24PM,245480,15,8467372,"Mizner Bioscience, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,245477,Released,1
14,245479,Released,1
15,245480,Released,4
16,245481,Released,1
17,245483,Released,25


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245477,NaN,NaN,1.0
245479,NaN,NaN,1.0
245480,NaN,NaN,4.0
245481,NaN,NaN,1.0
245483,NaN,NaN,25.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245403,14.0,8.0,0.0
245411,0.0,0.0,1.0
245415,0.0,1.0,0.0
245419,1.0,0.0,29.0
245422,0.0,2.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245403,14,8,0
245411,0,0,1
245415,0,1,0
245419,1,0,29
245422,0,2,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245403,14,8,0
1,245411,0,0,1
2,245415,0,1,0
3,245419,1,0,29
4,245422,0,2,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245403,14,8,
1,245411,,,1
2,245415,,1,
3,245419,1,,29
4,245422,,2,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation
7,Aug 31 2022 5:24PM,245480,15,"Mizner Bioscience, LLC"
11,Aug 31 2022 5:10PM,245483,15,"Virtus Pharmaceuticals, LLC"
36,Aug 31 2022 5:08PM,245479,15,"Carwin Pharmaceutical Associates, LLC"
37,Aug 31 2022 5:01PM,245481,102,"Senseonics, Incorporated"
38,Aug 31 2022 4:43PM,245477,12,Hush Hush
39,Aug 31 2022 3:55PM,245475,19,ACG North America LLC
42,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample
43,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.
49,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,,,7
1,Aug 31 2022 5:24PM,245480,15,"Mizner Bioscience, LLC",,,4
2,Aug 31 2022 5:10PM,245483,15,"Virtus Pharmaceuticals, LLC",,,25
3,Aug 31 2022 5:08PM,245479,15,"Carwin Pharmaceutical Associates, LLC",,,1
4,Aug 31 2022 5:01PM,245481,102,"Senseonics, Incorporated",,,1
5,Aug 31 2022 4:43PM,245477,12,Hush Hush,,,1
6,Aug 31 2022 3:55PM,245475,19,ACG North America LLC,,,3
7,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample,,,1
8,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,,2,4
9,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7,,
1,Aug 31 2022 5:24PM,245480,15,"Mizner Bioscience, LLC",4,,
2,Aug 31 2022 5:10PM,245483,15,"Virtus Pharmaceuticals, LLC",25,,
3,Aug 31 2022 5:08PM,245479,15,"Carwin Pharmaceutical Associates, LLC",1,,
4,Aug 31 2022 5:01PM,245481,102,"Senseonics, Incorporated",1,,
5,Aug 31 2022 4:43PM,245477,12,Hush Hush,1,,
6,Aug 31 2022 3:55PM,245475,19,ACG North America LLC,3,,
7,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample,1,,
8,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,4,2,
9,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7,,
1,Aug 31 2022 5:24PM,245480,15,"Mizner Bioscience, LLC",4,,
2,Aug 31 2022 5:10PM,245483,15,"Virtus Pharmaceuticals, LLC",25,,
3,Aug 31 2022 5:08PM,245479,15,"Carwin Pharmaceutical Associates, LLC",1,,
4,Aug 31 2022 5:01PM,245481,102,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7.0,NaN,NaN
1,Aug 31 2022 5:24PM,245480,15,"Mizner Bioscience, LLC",4.0,NaN,NaN
2,Aug 31 2022 5:10PM,245483,15,"Virtus Pharmaceuticals, LLC",25.0,NaN,NaN
3,Aug 31 2022 5:08PM,245479,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
4,Aug 31 2022 5:01PM,245481,102,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7.0,0.0,0.0
1,Aug 31 2022 5:24PM,245480,15,"Mizner Bioscience, LLC",4.0,0.0,0.0
2,Aug 31 2022 5:10PM,245483,15,"Virtus Pharmaceuticals, LLC",25.0,0.0,0.0
3,Aug 31 2022 5:08PM,245479,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
4,Aug 31 2022 5:01PM,245481,102,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,736355,9.0,0.0,0.0
102,245481,1.0,0.0,0.0
12,245477,1.0,0.0,0.0
15,981861,59.0,0.0,1.0
19,245475,3.0,0.0,0.0
20,490825,4.0,10.0,14.0
21,245415,0.0,1.0,0.0
22,490861,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,736355,9.0,0.0,0.0
1,102,245481,1.0,0.0,0.0
2,12,245477,1.0,0.0,0.0
3,15,981861,59.0,0.0,1.0
4,19,245475,3.0,0.0,0.0
5,20,490825,4.0,10.0,14.0
6,21,245415,0.0,1.0,0.0
7,22,490861,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,9.0,0.0,0.0
1,102,1.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,59.0,0.0,1.0
4,19,3.0,0.0,0.0
5,20,4.0,10.0,14.0
6,21,0.0,1.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,59.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,1.0,0.0,14.0,0.0,0.0
Executing,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0
Released,9.0,1.0,1.0,59.0,3.0,4.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,19,20,21,22
0,Completed,0.0,0.0,0.0,1.0,0.0,14.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0
2,Released,9.0,1.0,1.0,59.0,3.0,4.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,19,20,21,22
0,Completed,0.0,0.0,0.0,1.0,0.0,14.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0
2,Released,9.0,1.0,1.0,59.0,3.0,4.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()